In [ ]:
pip install yfinance

In [ ]:

import datetime
import time
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
import yfinance as yf
data = yf.download("EURUSD=X", period='60d',interval='5m')
period = "valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max"
interval = "valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo"

In [ ]:
#virtualize the closing price history
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(data['Close'])
plt.xlabel('Time(5m)')
plt.ylabel('Price in USD($)')
plt.show();

In [ ]:
#create dataframe with only the close price
df = data.filter(['Close'])
#convert dataframe to a numpy array
dataset = df.values
#Get the no of rows to train the model on
training_data_len = math.ceil(len(dataset)*0.8)

In [ ]:
#Scale the data(between 0 to 1)
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

In [ ]:
#create the train data set
train_data = scaled_data[0:training_data_len]
#split the data into x_train and y_train data set
x_train = []
y_train = []

#x_train = 60 values, y_train = 61st value
for i in range(60,len(train_data)):
  x_train.append(train_data[i-60:i])
  y_train.append(train_data[i])

#convert x_train and y_train into numpy arrrays
x_train,y_train = np.array(x_train),np.array(y_train)

#Reshape x_train to 3 dim because LSTM requires 3 dim
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

In [ ]:
#Build the LSTM Model

model = Sequential()
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50, return_sequences=False, input_shape = (x_train.shape[1],1)))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
#compile the model
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
#train the model
model.fit(x_train,y_train,batch_size=1,epochs=65)

In [ ]:
#create the test data set
test_data = scaled_data[training_data_len-60:]
#split the data into x_train and y_train data set
x_test = []
y_test = dataset[training_data_len:]

#x_train = 60 values, y_train = 61st value
for i in range(60,len(test_data)):
  x_test.append(test_data[i-60:i])

#convert x_train and y_train into numpy arrrays
x_test = np.array(x_test)

#Reshape x_train to 3 dim because LSTM requires 3 dim
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

In [ ]:
#Predict

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
#Evaluate

rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

In [ ]:
# Virtualize the train,test and valid

#Plot the data
train = data[0:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

#virtualize
plt.figure(figsize=(16,8))
plt.title('Model')
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Train','val','Predicts'],loc='upper right')
plt.xlabel('Time(5m)')
plt.ylabel('Price in USD($)')
plt.show();

In [ ]:
#show value and predicted price
valid[['Close','Predictions']].head()

In [ ]:
#Using the model in real time

current_data = yf.download("EURUSD=X", period='10d',interval='5m')
current_df = current_data.filter(["Close"])
# DF-->array
last_60_5m_data = current_df[-60:].values
# Scale(0,1)
last_60_5m_data_scaled = scaler.fit_transform(last_60_5m_data)
#create x_test
X_test = []
X_test.append(last_60_5m_data_scaled)
# X_test-->numpy array
X_test = np.array(X_test)
# Reshape
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

#Predict
current_predict = model.predict(X_test)
current_predict = scaler.inverse_transform(current_predict)

print(current_predict)

print(last_60_5m_data[-1])

if last_60_5m_data[-1] < current_predict:
  print("Green")
else:
  print('Red')